In [ ]:
# default_exp core

# core

> A small package with this one module, that will save you time writing validation code.  

TODO: Write more in here

Package only has the logic to deal with primative types and List, Tuple, Dict, and Union.  
This

---
**NOTES:**

This package comes with no warrenty and no garantees, it may or may not 

---
**LIMITATIONS**
1) 
1) Only support Tuple definitions of fixed size

>`ex. Tuple[int, float, str] and not Tuple[int, ...]`

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from inspect import getmodule
from typing import List, Tuple, Dict, Union, TypeVar, Callable
import os

## Type Check

In [ ]:
#export

# TODO: Make better exception messages that show the level where things went wrong
# TODO: Look into making custom exceptions

def validate_type(p, p_type):  # subclasses always ok for now    
    if (getmodule(p_type).__name__ == "typing") and (p_type is not TypeVar):
        # TypeVar doesn't have .__origin__
        parent_type = p_type.__origin__  # (Union, List, Tuple, Dict)
        
        if parent_type is tuple:
            if not issubclass(type(p), parent_type):
                raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
            inner_types = p_type.__args__
            if len(inner_types) != len(p):
                raise ValueError(("tuple expected {} elements, but " +
                                  "received {}").format(len(inner_types), len(p)))
            for idx, child in enumerate(p):
                validate_type(child, inner_types[idx])
                
        if parent_type is list:
            if not issubclass(type(p), parent_type):
                raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
            inner_type = p_type.__args__[0]
            for child in p:
                validate_type(child, inner_type)
        
        if parent_type is dict:
            if not issubclass(type(p), parent_type):
                raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
            key_type, value_type = p_type.__args__
            for key in p:
                validate_type(key, key_type)
                validate_type(p[key], value_type)
            
        if parent_type is Union:
            allowed_types = p_type.__args__
            for allowed_type in allowed_types:
                this_type = allowed_type.__origin__ if (getmodule(allowed_type).__name__ == "typing") else \
                    allowed_type
                if issubclass(type(p), this_type):
                    validate_type(p, allowed_type)
                    break
            else:
                raise TypeError(("expected one of the following types {}, " +
                                 "but received {}").format(allowed_types, type(p)))
                
              
    if (getmodule(p_type).__name__ == "builtins") and not issubclass(type(p), p_type):
        raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
#     print("Good type")
        

In [ ]:
#export
def validate(p, p_type, p_name:str, p_restriction:Callable=None):  # subclasses always ok for now
    # 1) check type
    try:
        validate_type(p, p_type)
    except TypeError as error:
        raise TypeError("{} expected type {}, but received {}".format(p_name, p_type, p)) from error
    
    if p_restriction is not None:
        if not p_restriction(p):
            raise ValueError("{} does not meet the restrictions enforced by {}".format(p, p_restriction))

    return p

List, Union, Tuple, Any

In [ ]:
d = validate([("1", 2),("hey", 2)], List[Tuple[str, int]], "mylist")

## Common Restrictions

In [ ]:
# import os
# os.path.isfile(p)
# os.path.isdir(p)
# os.path.

is a file
  -> parent exists
  -> exists
  -> doesn't exists
  -> 
  
is a directory
  -> exists
  -> doesn't exist
  -> contains a file with specific extension
  -> parent exists
  -> 
  
is a tuple with values

base values

value(s) have no restrictions = None
value(s) fall within a range = '(2,100]'
value(s) fall within a set = {}

value(s) are directories (with above options)
value(s) are files (with above options)

more complicated restrictions can use a function
value(s) evaluate function to True
  